## Using Converse API

The following examples show how to interact with Nova via the Converse API ([API Docs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html))

In [ ]:
import boto3
import json
import base64

# Create a Bedrock Runtime client
client = boto3.client("bedrock-runtime", region_name="us-east-1")

PRO_MODEL_ID = "us.amazon.nova-pro-v1:0"
LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"
MICRO_MODEL_ID = "us.amazon.nova-micro-v1:0"


### **Single Turn**

In [ ]:

# Define your system prompt(s).
system = [
    {
        "text": "You are an experienced publisher. For each user topic, respond with 3 potential book titles"
    }
]

# Your user prompt
messages = [
    {"role": "user", "content": [{"text": "a child graduating from high school"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])

### **Multi Turn**

In [ ]:

# Define one or more messages using the "user" and "assistant" roles.
messages = [
    {"role": "user", "content": [{"text": "How many days are in a week?"}]},
    {"role": "assistant", "content": [{"text": "There are seven days in a week"}]},
    {"role": "user", "content": [{"text": "Which day is the first?"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])

### **Streaming**

In [ ]:

# Define your system prompt(s).
system = [
    {
        "text": "Act as a creative writing assistant. When the user provides you with a topic, provide a list of 3 potential titles for a short story based on that topic."
    }
]

messages = [
    {"role": "user", "content": [{"text": "A camping trip"}]},
]

# Configure the inference parameters.
inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse_stream(
    modelId=LITE_MODEL_ID, messages=messages, system=system, inferenceConfig=inf_params
)

stream = model_response.get("stream")
if stream:
    for event in stream:
        if "contentBlockDelta" in event:
            print(event["contentBlockDelta"]["delta"]["text"], end="")

### **Image Understanding**

In [ ]:

with open("media/sunset.png", "rb") as f:
    image = f.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "png", "source": {"bytes": image}}},
            {"text": "Describe the following image"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])

In [ ]:

# Open the image you'd like to use and encode it as a Base64 string.
with open("media/dog.jpeg", "rb") as image_file:
    dog_binary_data = image_file.read()

with open("media/cat.jpeg", "rb") as image_file:
    cat_binary_data = image_file.read()

messages = [
    {
        "role": "user",
        "content": [
            {"image": {"format": "jpeg", "source": {"bytes": dog_binary_data}}},
            {"image": {"format": "jpeg", "source": {"bytes": cat_binary_data}}},
            {"text": "What do these two images have in common"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print(model_response["output"]["message"]["content"][0]["text"])

### **Video Understanding**


In [ ]:

with open("./media/the-sea.mp4", "rb") as file:
    media_bytes = file.read()
    media_base64 = base64.b64encode(media_bytes)


messages = [
    {
        "role": "user",
        "content": [
            {"video": {"format": "mp4", "source": {"bytes": media_bytes}}},
            {"text": "Describe the following video"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])

### S3 Path for Video Understanding 

##### Replace the S3 bucket URI with URI where the video file is located

In [ ]:

messages = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": "mp4",
                    "source": {
                        "s3Location": {
                            #Replace the s3 bucket URI 
                            "uri": "s3://demo-bucket/the-sea.mp4"
                        }
                    },
                }
            },
            {"text": "Describe the following video"},
        ],
    }
]

inf_params = {"maxTokens": 300, "topP": 0.1, "temperature": 0.3}

model_response = client.converse(
    modelId=LITE_MODEL_ID, messages=messages, inferenceConfig=inf_params
)

print("\n[Full Response]")
print(json.dumps(model_response, indent=2))

print("\n[Response Content Text]")
print(model_response["output"]["message"]["content"][0]["text"])